### ООП - продвинутый (домашнее задание)


In [ ]:
from datetime import datetime
from pathlib import Path

class AbstractFile:

    def __init__(self, path: str, size: int, create_ts: datetime, update_ts: datetime, created_by: str, updated_by: str, content_type: str):
        self.path = path
        self.create_ts = create_ts
        self.update_ts = update_ts
        self.created_by = created_by
        self.updated_by = updated_by
        self.content_type = content_type

    def __repr__(self):
        return f"{type(self)}({self.path})"

    def touch(self):
        """Создаёт файл по пути path, если файл ещё не существует."""
        raise NotImplementedError()
    
    def read(self) -> bytes:
        """Читает файл. Возвращает содержимое файла в виде массива байтов."""
        raise NotImplementedError()

    def write(self, content: bytes) -> int:
        """Записывает файл. В качестве параметра принимает массив байтов. Возвращает количество записанных байтов."""
        raise NotImplementedError()
    
    def delete(self):
        """Удаляет файл."""
        raise NotImplementedError()


class LocalFile(AbstractFile):

    def __init__(self, path, size, create_ts, update_ts, created_by, updated_by, content_type):
        super().__init__(path, size, create_ts, update_ts, created_by, updated_by, content_type)
        self.path_obj = Path(self.path)

    def touch(self):
        self.path_obj.touch()
        self.create_ts = datetime.now()

    def read(self):
        return self.path_obj.read_bytes()
    
    def write(self, content):
        self.path_obj.write_bytes(content)
        self.update_ts = datetime.now()
        return len(content)
    
    def delete(self):
        if self.path_obj.is_dir():
            self.path_obj.rmdir()
        else:
            self.path_obj.unlink()

class FtpFile(AbstractFile):

    def __init__(self, path, size, create_ts, update_ts, created_by, updated_by, content_type):
        super().__init__(path, size, create_ts, update_ts, created_by, updated_by, content_type)

    def touch(self):
        """Создаёт файл по пути path на удалённом FTP-сервере"""
        raise NotImplementedError()

    def read(self):
        """Читает файл по пути path с удалённого FTP-сервера"""
        raise NotImplementedError()
    
    def write(self, content):
        """Записывает содержимое content в файл по пути path на удалённый FTP-сервер"""
        raise NotImplementedError()
        return len(content)
    
    def delete(self):
        """Удаляет файл по пути path с удалённого FTP-сервера"""
        raise NotImplementedError()


class AudioFile(AbstractFile):

    def __init__(self, path, size, create_ts, update_ts, created_by, updated_by, content_type, author: str, title: str, genre: str, release_date: datetime):
        super().__init__(path, size, create_ts, update_ts, created_by, updated_by, content_type)
        self.author = author
        self.title = title
        self.genre = genre
        self.release_date = release_date

    def __read_metadata(self, content):
        # self.author = <read author from file>
        # self.title = <read title from file>
        # self.genre = <read genre from file>
        # self.release_date = <read release_date from file>
        pass

    def read(self):
        content = super().read()
        self.__read_metadata(content)
        return content


class VideoFile(AbstractFile):

    def __init__(self, path, size, create_ts, update_ts, created_by, updated_by, content_type, title: str, release_date: datetime, width: int, height: int):
        super().__init__(path, size, create_ts, update_ts, created_by, updated_by, content_type)
        self.title = title
        self.release_date = release_date
        self.width = width
        self.height = height

    def __read_metadata(self, content):
        # self.title = <read title from file>
        # self.release_date = <read release_date from file>
        # self.width = <read width from file>
        # self.height = <read height from file>
        pass

    def read(self):
        content = super().read()
        self.__read_metadata(content)
        return content


class PhotoFile(AbstractFile):

    def __init__(self, path, size, create_ts, update_ts, created_by, updated_by, content_type, width: int, height: int, tags: list):
        super().__init__(path, size, create_ts, update_ts, created_by, updated_by, content_type)
        self.width = width
        self.height = height
        self.tags = tags

    def __read_metadata(self, content):
        # self.width = <read width from file>
        # self.height = <read height from file>
        # self.tags = <read tags from file>
        pass

    def read(self):
        content = super().read()
        self.__read_metadata(content)
        return content


Примеры работы с файлами

In [ ]:
### Чтение фотографии и её метаданных
photo_file = PhotoFile('~/Photos/photo.jpg')
photo_content = photo_file.read()
photo_width = photo_file.width
photo_height = photo_file.height
photo_tags = photo_file.tags

In [ ]:
### Отправка файла на FTP-сервер
ftp_file = FtpFile('ftp://remote.host/app/file.bin')
ftp_file.touch()
ftp_file.write('Hello, world'.encode())

##### Ответы на вопросы

1) много ли кода придется дописать / переписать при добавлении новых типов файлов?

При условии наследования типизированного файла от конкретной реализации хранения (например, локальный файл), потребуется дописать код только специфичный для типа/формата файла.
Например: работу с метаданными файла; особые операции кодирования/декодирования данных файла.

2) много ли кода придется дописать / переписать при добавлении новых способов хранения файлов?

Придётся написать код всех основных операций с файлом: создание, удаление, чтение, запись и т.д., т.к. они специфичны для способа хранения.
Базовые атрибуты/характеристики файла останутся неизменными: поле для хранения пути к файлу, его размер, тип контента, и др..
